In [1]:
# 4-DOF Human-Arm-Like Robot Simulator (DH Convention)
# ----------------------------------------------------
# Visualizes a 4-DOF robot arm using standard DH parameters in a Z-up world.
# World axes (X=red, Y=green, Z=blue) are fixed; the arm moves relative to them.
# Sliders control θ₁–θ₄ (deg) and link lengths L₁–L₃. View Pitch/Yaw are in degrees.
# Shows the end-effector transform T₀₄. Trace option records the end-effector path.
# Written by Dr. Harsha Abeykoon, with help from ChatGPT (2025).


from IPython.display import HTML
HTML(r"""
<style>
  .panel { font-family: system-ui, Segoe UI, Roboto, Arial; margin: 8px 0; }
  .card  { display:flex; gap:16px; flex-wrap:wrap; align-items:flex-start; }
  .sidebar { min-width:360px; max-width:460px; }
  .matrix { font-family: ui-monospace, SFMono-Regular, Menlo, Consolas, monospace;
            white-space: pre; background:#fafafa; border:1px solid #eee; border-radius:8px; padding:8px; }
  canvas { border:1px solid #ddd; border-radius:8px; background:#fff; }
  input[type=number]{width:90px;}
  .note { color:#555; font-size:12px; }
</style>

<div class="panel">
  <h3>4-DOF Human-Arm-Like Robot — by  Harsha Abeykoon</h3>
  <div class="note">Z-up world frame: X=red, Y=green, Z=blue (fixed). DH convention used.</div>
  <div class="card">
    <canvas id="view3d" width="740" height="520"></canvas>

    <div class="sidebar">
      <label>θ₁ (deg): <span id="th1Lbl">30.0</span></label>
      <input id="th1" type="range" min="-180" max="180" step="0.1" value="30" style="width:100%">
      <br>
      <label>θ₂ (deg): <span id="th2Lbl">20.0</span></label>
      <input id="th2" type="range" min="-180" max="180" step="0.1" value="20" style="width:100%">
      <br>
      <label>θ₃ (deg): <span id="th3Lbl">40.0</span></label>
      <input id="th3" type="range" min="-180" max="180" step="0.1" value="40" style="width:100%">
      <br>
      <label>θ₄ (deg): <span id="th4Lbl">-15.0</span></label>
      <input id="th4" type="range" min="-180" max="180" step="0.1" value="-15" style="width:100%">
      <br><br>

      <label>L1:</label> <input id="L1" type="number" value="0.9" step="0.01">
      &nbsp;&nbsp;<label>L2:</label> <input id="L2" type="number" value="0.8" step="0.01">
      &nbsp;&nbsp;<label>L3:</label> <input id="L3" type="number" value="0.25" step="0.01">
      <br><br>

      <label>View Pitch (deg): <span id="rxLbl">-90</span></label>
      <input id="rx" type="range" min="-180" max="180" step="1" value="-90" style="width:100%">
      <br>
      <label>View Yaw (deg): <span id="ryLbl">45</span></label>
      <input id="ry" type="range" min="-180" max="180" step="1" value="45" style="width:100%">
      <br><br>

      <button id="resetBtn">Reset</button>
      <label style="margin-left:8px;"><input id="traceChk" type="checkbox" checked> Show trace</label>
      <br><br>

      <div><b>T (Homogeneous Transform)</b></div>
      <div id="matrix" class="matrix"></div>
    </div>
  </div>
</div>

<script>

(() => {
  const cos=Math.cos,sin=Math.sin,d2r=d=>d*Math.PI/180,r2d=r=>r*180/Math.PI;

  // ---- DH transform generator ----
  function dh(a, alpha, d, theta){
    const ca = cos(alpha), sa = sin(alpha);
    const ct = cos(theta), st = sin(theta);
    return [
      ct, -st*ca, st*sa, a*ct,
      st,  ct*ca, -ct*sa, a*st,
      0,   sa,     ca,    d,
      0,   0,      0,     1
    ];
  }

  function mul(A,B){
    const C=new Array(16).fill(0);
    for(let r=0;r<4;r++)for(let c=0;c<4;c++)
      for(let k=0;k<4;k++) C[r*4+c]+=A[r*4+k]*B[k*4+c];
    return C;
  }
  function apply(T,p){return[
    T[0]*p[0]+T[1]*p[1]+T[2]*p[2]+T[3],
    T[4]*p[0]+T[5]*p[1]+T[6]*p[2]+T[7],
    T[8]*p[0]+T[9]*p[1]+T[10]*p[2]+T[11],1];}

  // ---- Canvas & Camera ----
  const canvas=document.getElementById('view3d');
  const ctx=canvas.getContext('2d');

  // Camera uses radians internally; sliders are degrees
  const cam={
    rotX: d2r(parseFloat(document.getElementById('rx').value)), // pitch (rad)
    rotY: d2r(parseFloat(document.getElementById('ry').value)), // yaw (rad)
    dist: 4.0,
    fov: 800.0
  };

  function Rx(a){const c=cos(a),s=sin(a);
    return[1,0,0,0, 0,c,-s,0, 0,s,c,0, 0,0,0,1];}
  function Ry(a){const c=cos(a),s=sin(a);
    return[ c,0,s,0, 0,1,0,0, -s,0,c,0, 0,0,0,1];}

  function projPoint(p){
    const Tcam=mul(Ry(cam.rotY),Rx(cam.rotX));
    const pw=apply(Tcam,[p[0],p[1],p[2],1]);
    const z=pw[2]+cam.dist;
    const scale=cam.fov/(cam.fov+z*120);
    return {X:canvas.width/2+pw[0]*140*scale,
            Y:canvas.height/2-pw[1]*140*scale};
  }

  function drawAxes(l=0.5){
    const O=projPoint([0,0,0]),
          X=projPoint([ l, 0, 0]),
          Y=projPoint([ 0, l, 0]),
          Z=projPoint([ 0, 0, l]);
    ctx.lineWidth=2;
    ctx.strokeStyle="#c33"; ctx.beginPath(); ctx.moveTo(O.X,O.Y); ctx.lineTo(X.X,X.Y); ctx.stroke();
    ctx.strokeStyle="#3a3"; ctx.beginPath(); ctx.moveTo(O.X,O.Y); ctx.lineTo(Y.X,Y.Y); ctx.stroke();
    ctx.strokeStyle="#36c"; ctx.beginPath(); ctx.moveTo(O.X,O.Y); ctx.lineTo(Z.X,Z.Y); ctx.stroke();
    ctx.fillStyle="#c33"; ctx.fillText("X", X.X+8, X.Y);
    ctx.fillStyle="#3a3"; ctx.fillText("Y", Y.X+8, Y.Y);
    ctx.fillStyle="#36c"; ctx.fillText("Z", Z.X+8, Z.Y);
  }

  // ---- Elements & state ----
  const el=id=>document.getElementById(id);
  const th1=el('th1'), th2=el('th2'), th3=el('th3'), th4=el('th4');
  const th1Lbl=el('th1Lbl'), th2Lbl=el('th2Lbl'), th3Lbl=el('th3Lbl'), th4Lbl=el('th4Lbl');
  const L1in=el('L1'), L2in=el('L2'), L3in=el('L3');
  const rx=el('rx'), ry=el('ry'), rxLbl=el('rxLbl'), ryLbl=el('ryLbl');
  const resetBtn=el('resetBtn'), traceChk=el('traceChk'), matDiv=el('matrix');
  let trace=[];

  // Capture DOM defaults so Reset follows the HTML values
  const defaults = {
    th1: parseFloat(th1.value),
    th2: parseFloat(th2.value),
    th3: parseFloat(th3.value),
    th4: parseFloat(th4.value),
    L1 : parseFloat(L1in.value),
    L2 : parseFloat(L2in.value),
    L3 : parseFloat(L3in.value),
    rx : parseFloat(rx.value),  // degrees
    ry : parseFloat(ry.value)   // degrees
  };

  function fmt4(n){return (Math.abs(n)<1e-12?0:n).toFixed(4);}

  function render(){
    const t1=d2r(parseFloat(th1.value)), t2=d2r(parseFloat(th2.value)),
          t3=d2r(parseFloat(th3.value)), t4=d2r(parseFloat(th4.value));
    th1Lbl.textContent=(+th1.value).toFixed(1);
    th2Lbl.textContent=(+th2.value).toFixed(1);
    th3Lbl.textContent=(+th3.value).toFixed(1);
    th4Lbl.textContent=(+th4.value).toFixed(1);
    const L1=parseFloat(L1in.value), L2=parseFloat(L2in.value), L3=parseFloat(L3in.value);

    // DH parameters (a, alpha, d, theta)
    const T01 = dh(0,  Math.PI/2, 0, t1);
    const T12 = dh(L1, 0, 0, t2);
    const T23 = dh(L2, 0, 0, t3);
    const T34 = dh(L3, 0, 0, t4);

    const T02 = mul(T01, T12);
    const T03 = mul(T02, T23);
    const T04 = mul(T03, T34);

    const p1 = apply(T01,[0,0,0,1]);
    const p2 = apply(T02,[0,0,0,1]);
    const p3 = apply(T03,[0,0,0,1]);
    const p4 = apply(T04,[0,0,0,1]);

    if (traceChk.checked){ trace.push([p4[0],p4[1],p4[2]]); if(trace.length>2500) trace.shift(); }
    else trace=[];

    // Draw grid
    ctx.clearRect(0,0,canvas.width,canvas.height);
    ctx.strokeStyle="#f0f0f0"; ctx.lineWidth=1;
    for(let x=-2.5;x<=2.5;x+=0.25){
      let a=projPoint([x,-2.5,0]), b=projPoint([x, 2.5,0]);
      ctx.beginPath(); ctx.moveTo(a.X,a.Y); ctx.lineTo(b.X,b.Y); ctx.stroke();
    }
    for(let y=-2.5;y<=2.5;y+=0.25){
      let a=projPoint([-2.5,y,0]), b=projPoint([ 2.5,y,0]);
      ctx.beginPath(); ctx.moveTo(a.X,a.Y); ctx.lineTo(b.X,b.Y); ctx.stroke();
    }
    drawAxes(0.5);

    // Draw robot
    const P0=projPoint([0,0,0]),
          P1=projPoint([p1[0],p1[1],p1[2]]),
          P2=projPoint([p2[0],p2[1],p2[2]]),
          P3=projPoint([p3[0],p3[1],p3[2]]),
          P4=projPoint([p4[0],p4[1],p4[2]]);

    ctx.lineWidth=5; ctx.strokeStyle="#999";
    ctx.beginPath(); ctx.moveTo(P0.X,P0.Y); ctx.lineTo(P1.X,P1.Y);
    ctx.lineTo(P2.X,P2.Y); ctx.lineTo(P3.X,P3.Y); ctx.lineTo(P4.X,P4.Y); ctx.stroke();

    ctx.fillStyle="#444";
    [[P0.X,P0.Y],[P1.X,P1.Y],[P2.X,P2.Y],[P3.X,P3.Y],[P4.X,P4.Y]].forEach(([X,Y])=>{
      ctx.beginPath(); ctx.arc(X,Y,4,0,2*Math.PI); ctx.fill();
    });

    if (trace.length>1){
      ctx.strokeStyle="#999"; ctx.lineWidth=1.2;
      ctx.beginPath();
      let q=projPoint(trace[0]); ctx.moveTo(q.X,q.Y);
      for(let i=1;i<trace.length;i++){ q=projPoint(trace[i]); ctx.lineTo(q.X,q.Y); }
      ctx.stroke();
    }

    // Show T04
    const M = T04;
    const row=r=>`[ ${fmt4(M[r*4+0])}  ${fmt4(M[r*4+1])}  ${fmt4(M[r*4+2])}  ${fmt4(M[r*4+3])} ]`;
    matDiv.textContent = row(0)+"\n"+row(1)+"\n"+row(2)+"\n"+row(3);
  }

  // ---- Events ----
  ['th1','th2','th3','th4','L1','L2','L3'].forEach(id=>{
    el(id).addEventListener('input',render);
    el(id).addEventListener('change',render);
  });

  // View sliders are in degrees; convert to radians for cam
  rx.addEventListener('input',()=>{
    const deg = parseFloat(rx.value);
    rxLbl.textContent = deg.toFixed(0);
    cam.rotX = d2r(deg);
    render();
  });
  ry.addEventListener('input',()=>{
    const deg = parseFloat(ry.value);
    ryLbl.textContent = deg.toFixed(0);
    cam.rotY = d2r(deg);
    render();
  });

  function reset(){
    // Restore slider values from HTML defaults captured at load
    th1.value=defaults.th1; th2.value=defaults.th2;
    th3.value=defaults.th3; th4.value=defaults.th4;
    L1in.value=defaults.L1; L2in.value=defaults.L2; L3in.value=defaults.L3;

    // Reset view sliders (degrees) and camera (radians)
    rx.value=defaults.rx; ry.value=defaults.ry;
    rxLbl.textContent = (+defaults.rx).toFixed(0);
    ryLbl.textContent = (+defaults.ry).toFixed(0);
    cam.rotX = d2r(defaults.rx);
    cam.rotY = d2r(defaults.ry);

    // Update theta labels too
    th1Lbl.textContent=(+th1.value).toFixed(1);
    th2Lbl.textContent=(+th2.value).toFixed(1);
    th3Lbl.textContent=(+th3.value).toFixed(1);
    th4Lbl.textContent=(+th4.value).toFixed(1);

    trace=[]; render();
  }
  resetBtn.addEventListener('click',reset);

  // Initial render
  // Ensure labels match initial slider degree values
  rxLbl.textContent = (+rx.value).toFixed(0);
  ryLbl.textContent = (+ry.value).toFixed(0);
  render();
})();
</script>
""")
